In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *
from hard_hfs import *
import copy

In [3]:
def RMSE(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        for j in range(len(ground[0])):
            if ground[i,j] != 0:
                error += (ground[i,j] - predict[i,j])**2
                n += 1
                
    return np.sqrt(error/n)

In [4]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/home/marc/Documents/MVA/ProjetGraphes/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5]), 'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.]), 'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.])}


In [5]:
P_dict = copy.deepcopy(R_dict)
P_dict["Ratings"] = np.ones([len(R_dict["Ratings"])])
P = R > 0
print(P)

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [ True False False ..., False False False]
 [ True False False ..., False False False]]


In [6]:
max([np.sum(P[i,:] for i in range(len(P)))][0])

341

In [7]:
len(P[0])

9066

In [56]:
N = len(R)
M = len(R[0])
K = 4

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als.fit(R_dict)

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [57]:
als.U

array([[-0.43055943,  0.01198527, -0.60762093,  1.31850984],
       [ 0.14243279,  0.66327219, -0.80479377,  1.46866163],
       [-0.31272574,  0.07217373, -0.60637123,  1.70456461],
       ..., 
       [-0.17201845,  0.65367101, -0.25615963,  1.70854941],
       [ 0.4016926 ,  0.89576162, -0.4089591 ,  1.53208619],
       [-0.24608042,  1.02800234, -0.6423355 ,  1.39951243]])

In [58]:
als.V

array([[-0.08733414,  1.13441725, -0.56358884,  1.71848785],
       [-0.24468365,  0.67746976, -0.99329832,  1.39433299],
       [ 0.35518313,  0.6096451 , -1.23307834,  1.13264668],
       ..., 
       [-1.24758967,  0.66333255, -1.3425005 ,  1.29248794],
       [-0.7485538 ,  0.39799953, -0.8055003 ,  0.77549276],
       [-0.59328318,  1.11132786,  0.50315819,  2.19262199]])

In [59]:
R_rec = np.dot(als.U,np.transpose(als.V))
print((R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec)))
print(R_rec)

[[ 3.23759914  3.20274249  3.04913762 ...,  3.373472    2.96268103
   3.30278189]
 [ 3.59208138  3.43937892  3.38881724 ...,  3.43242171  2.99805085
   3.50846668]
 [ 3.47908585  3.38669082  3.22145157 ...,  3.50421816  3.04112873
   3.58560478]
 ..., 
 [ 3.63215886  3.39245717  3.21379828 ...,  3.41883404  2.98989825
   3.83613486]
 [ 3.63463613  3.36878755  3.32768889 ...,  3.22512821  2.87367476
   3.65683086]
 [ 3.67159082  3.46764841  3.3237197  ...,  3.57287049  3.08232012
   3.69809967]]
[[ 2.65949034  2.55546113  2.09702932 ...,  3.06500118  1.83900071
   2.85402745]
 [ 3.71743813  3.26169955  3.11079886 ...,  3.24093585  1.94456151
   3.4678911 ]
 [ 3.38020438  3.10445259  2.61129813 ...,  3.45521147  2.07312688
   3.6981088 ]
 ..., 
 [ 3.83704886  3.12166218  2.58845699 ...,  3.20038363  1.92023018
   4.44581265]
 [ 3.84444225  3.05102051  2.92836207 ...,  2.62227102  1.57336261
   3.91068214]
 [ 3.95473304  3.34606949  2.91651608 ...,  3.66010348  2.19606209
   4.03384838]]


In [60]:
print(RMSE(R,R_rec))
print(RMSE(R,(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))))

0.699005156336
0.888810201158


In [37]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [38]:
np.max(R_rec)

1.6487915739788013

In [39]:
lp = LaplacianParams()

# sim = similarity(als.U)
sim = build_graph(als.U, GraphParams())
# Seems to work better with U... 

print(sim)

[[ 0.          0.          0.         ...,  0.          0.99590678  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.99590678  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [40]:
test = np.asarray([np.sum([sim[0,j]*P[j,i] for j in range(1,len(P))])/np.sum([sim[0,j] for j in range(1,len(P))]) for i in range(len(P[0]))])

In [41]:
np.mean(np.sqrt((test - P[0])**2))

0.0044567704885354192

In [42]:
np.max(test)

0.40037620986575145

In [43]:
test[P[0]>0]

array([ 0.19977619,  0.        ,  0.0998754 ,  0.0998754 ,  0.19977012,
        0.        ,  0.        ,  0.        ,  0.19977619,  0.10017068,
        0.19977619,  0.        ,  0.        ,  0.        ,  0.        ,
        0.0998754 ,  0.        ,  0.20000202,  0.        ,  0.        ])

In [44]:
L = build_laplacian(sim,lp)

print(L.shape)

(671, 671)


In [45]:
np.sum(P[:,1] == 1)

107

In [46]:
np.sum(P[:100,1] == 1)

20

In [47]:
test_vec = copy.deepcopy(P[:,1]) + 1
test_vec[:100] = [0 for i in range(100)]
test_vec.shape

(671,)

In [48]:
hfs0 = simple_hfs(als.U, test_vec, L, sim)
hfs0

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  2.,  2.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  2.,  1.,  1.,
        1.,  2.,  1.,  1.,  1.,  1.,  2.,  1.,  2.,  1.,  2.,  1.,  1.,
        1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  1.,  1.,  1.,  1.,  1.,
        2.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  1.,  1.,  1.,  1

In [49]:
np.sum(hfs0[:100] == 2)

7

In [50]:
np.sum((hfs0[:100] - P[:100,1]*2)==0)

5

In [149]:
np.sum(hfs0.argmax(axis=0))

14

In [ ]:
hfs0[0][P[0]==0]

In [ ]:
np.min(hfs0[0][P[0]!=0])

In [ ]:
hfs0